In [36]:
import pandas as pd
import numpy as np
from typing import Tuple
from scipy.stats import mode
from sklearn.metrics import confusion_matrix

In [37]:
def load_csv(csv_path:str) ->Tuple[pd.DataFrame,pd.DataFrame]:
        df = pd.read_csv(csv_path,na_values='""',delimiter=',',header=None)
        df = df.sample(frac=1,random_state=42).reset_index(drop=True)
        x,y = df.iloc[:,:4],df.iloc[:,-1]
        y = pd.DataFrame(y).reset_index(drop=True)
        return x,y

x,y = load_csv('iris.csv')
#load_csv('iris.csv')
#x,y
# isinstance(x,pd.DataFrame)
#isinstance(y,pd.DataFrame)
#x,y

cleanup

In [38]:
x[pd.isnull(x)] = 3.5

y = y[x.min(axis=1) >= 0]
x = x[x.min(axis=1) >= 0]
y = y[x.max(axis=1) < 13]
x = x[x.max(axis=1) < 13]
#y = np.delete(y,np.where(x > 13.0)[0],axis=0)
#x = np.delete(x,np.where(x < 0.0)[0],axis=0)
#x = np.delete(x,np.where(x > 13.0)[0],axis=0)
#x.shape,y.shape


In [39]:
def train_test_split(features:pd.DataFrame,labels:pd.DataFrame,test_split_ratio:float) -> None:
        test_size = int(len(features) * test_split_ratio)
        train_size = len(features) - test_size
        assert len(features) == test_size + train_size, "Size mismatch!"

        x_train,y_train = features.iloc[:train_size,:],labels.iloc[:train_size]
        x_test,y_test = features.iloc[train_size:train_size+test_size,:], labels.iloc[train_size:train_size + test_size]
        return (x_train,y_train,x_test,y_test)
        #self.x_train = x_train
        #self.y_train = y_train
        #self.x_test = x_test
        #self.y_test = y_test

x_train,y_train,x_test,y_test =train_test_split(x,y,0.15)
#x_train.shape,y_train.shape,x_test.shape,y_test.shape

In [40]:
def euclidean(x_train,element_of_x:pd.DataFrame) -> pd.DataFrame:
        return pd.DataFrame(((x_train - element_of_x)**2).sum(axis=1).pow(1./2))



In [48]:
def predict(k:int,x_test:pd.DataFrame,x_train:pd.DataFrame,y_train:pd.DataFrame) -> pd.DataFrame:
        labels_pred = []
        for idx,x_test_element in x_test.iterrows():
            distances = euclidean(x_train,x_test_element)
            distances['y']=y_train
            distances.sort_values(by=[0],inplace=True)
            label_pred = distances.head(k)['y'].mode()
            #print(label_pred)
            labels_pred.append(label_pred)

        y_preds = pd.DataFrame(labels_pred)
        return y_preds

ypreds=predict(3,x_test,x_train,y_train)
#print(ypreds.shape)

In [49]:
def accuracy(y_test,y_preds) -> float:
        y_test.reset_index(drop=True,inplace=True)
        y_preds.reset_index(drop=True,inplace=True)
        y_test.columns=['y']
        y_preds.columns=['y']
        true_positive = (y_test == y_preds).sum()
        return true_positive / len(y_test) * 100



accuracy(y_test,ypreds)

y    85.714286
dtype: float64

In [50]:
def plot_confusion_matrix(y_test:pd.DataFrame,y_preds:pd.DataFrame):
        
        conf_matrix = confusion_matrix(y_test,y_preds)
        return conf_matrix




cf = plot_confusion_matrix(y_test,ypreds)
print(cf)

[[5 0 0]
 [0 6 0]
 [0 3 7]]
